In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float32)
print(f"Using device: {device}")

Using device: cuda


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1)),  # flatten 32x32x3 to 3072
    transforms.Lambda(lambda x: x.to(torch.float32))  # ensure cons dtype
])

trainset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

batch_size = 128
trainloader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True  # workers alive between epochs
)

100%|██████████| 170M/170M [00:05<00:00, 30.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [3]:
class VAE(nn.Module):
    def __init__(self, input_dim=3072, hidden_dim=400, latent_dim=20):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )

        self.fc_mu = nn.Linear(hidden_dim, latent_dim) # mean
        self.fc_var = nn.Linear(hidden_dim, latent_dim) # std

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.fc_mu(h), self.fc_var(h)

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        return self.decoder(z), mu, log_var

In [ ]:
model = VAE().to(device)
test_batch, _ = next(iter(trainloader))
test_batch = test_batch.to(device)
recon_batch, mu, log_var = model(test_batch)

print(f"input shape: {test_batch.shape}")
print(f"reconstructed shape: {recon_batch.shape}")
print(f"mu shape: {mu.shape}")
print(f"log_var shape: {log_var.shape}")

input shape: torch.Size([128, 3072])
reconstructed shape: torch.Size([128, 3072])
mu shape: torch.Size([128, 20])
log_var shape: torch.Size([128, 20])


In [4]:
def loss_function(recon_x, x, mu, log_var):
    # reconstruction loss (BCE)
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # kl divergence loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    return BCE + KLD

In [5]:
def train(model, trainloader, epochs=30):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (data, _) in enumerate(tqdm(trainloader)):
            data = data.to(device)
            optimizer.zero_grad()

            recon_batch, mu, log_var = model(data)

            loss = loss_function(recon_batch, data, mu, log_var)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(trainloader.dataset)
        print(f"epoch [{epoch+1}/{epochs}] avg Loss: {avg_loss:.4f}")

In [6]:
def plot_reconstruction(model, data):
    model.eval()
    with torch.no_grad():

        recon_batch, _, _ = model(data.to(device))

        # plot original and reconstruction
        plt.figure(figsize=(12, 6))

        for i in range(8):
            plt.subplot(2, 8, i + 1)
            plt.imshow(data[i].view(3, 32, 32).permute(1, 2, 0).cpu())
            plt.axis('off')
            if i == 0:
                plt.title("original")

        for i in range(8):
            plt.subplot(2, 8, i + 9)
            plt.imshow(recon_batch[i].view(3, 32, 32).permute(1, 2, 0).cpu())
            plt.axis('off')
            if i == 0:
                plt.title("reconstruction")

        plt.tight_layout()

In [ ]:
model = VAE().to(device)
train(model, trainloader, epochs=30)

test_batch, _ = next(iter(trainloader))
plot_reconstruction(model, test_batch)

In [ ]:
def generate_images(model, num_images=8):
    model.eval()
    with torch.no_grad():

        z = torch.randn(num_images, 20).to(device)
        sample = model.decoder(z)

        plt.figure(figsize=(12, 3))
        for i in range(num_images):
            plt.subplot(1, num_images, i + 1)
            plt.imshow(sample[i].view(3, 32, 32).permute(1, 2, 0).cpu())
            plt.axis('off')
        plt.tight_layout()

generate_images(model)